# Task 2: RoBERTa based ADR Classification

## 1. Install necessary libraries

In [ ]:
!pip install -q contractions transformers sent2vec imbalanced-learn seqeval[gpu] ekphrasis
!pip install -q tf-estimator-nightly==2.8.0.dev2021122109
# !python -m pip uninstall -q -y spacy
# !python -m pip install -q -U spacy

     |████████████████████████████████| 4.0 MB 9.8 MB/s 
     |████████████████████████████████| 43 kB 1.0 MB/s 
     |████████████████████████████████| 80 kB 2.9 MB/s 
     |████████████████████████████████| 106 kB 38.6 MB/s 
     |████████████████████████████████| 287 kB 18.2 MB/s 
     |████████████████████████████████| 77 kB 3.3 MB/s 
     |████████████████████████████████| 895 kB 39.2 MB/s 
     |████████████████████████████████| 6.6 MB 9.0 MB/s 
     |████████████████████████████████| 596 kB 28.1 MB/s 
     |████████████████████████████████| 45 kB 1.1 MB/s 
     |████████████████████████████████| 53 kB 1.2 MB/s 
     |████████████████████████████████| 462 kB 7.8 MB/s 
     |████████████████████████████████| 6.0 MB 11.1 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 653 kB 40.0 MB/s 
     |████████████████████████████████| 457 kB 47.6 MB/s 
     |████████████████████████████████| 10.1 MB 40.9 MB/s 
     |██████████████████████

In [ ]:
# !python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 409 kB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## 2. Load all the libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import warnings
import torch.nn as nn

from transformers import RobertaTokenizerFast, RobertaModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, classification_report, f1_score
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from tqdm import tqdm

pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth=None
warnings.filterwarnings("ignore")

## 3. Load the dataset

In [ ]:
# Load data
training = pd.read_csv('task3_training.tsv',sep='\t', usecols=['tweet_id', 'begin', 'end', 'type', 'extraction', 'drug', 'tweet', 'meddra_code', 'meddra_term'])
validation = pd.read_csv('task3_validation.tsv', sep="\t", skipinitialspace=True)

In [ ]:
training.head()

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,331187619096588288,NaN,NaN,NaN,NaN,ofloxacin,@seefisch:oral drugs for pyelonephritis:ciprofloxacin levofloxacin tmp/smz do not use nitrofurantoin for pyelo(only cystitis)@david_medinaf,NaN,NaN
1,332227554956161024,NaN,NaN,NaN,NaN,trazodone,happy for wellbutrin; has similar effects as adderall.. trazodone is super promising for sleep.. but abilify can cause weight gain -_-,NaN,NaN
2,332448217490944000,NaN,NaN,NaN,NaN,lamotrigine,"@stilgarg i'm ok ty have an official diagnosis of bipolar now, feeling ok at the moment lamotrigine has been increased having monotherapy:/",NaN,NaN
3,332977955754110976,NaN,NaN,NaN,NaN,cymbalta,i'm soo depressed cymbalta couldn't help me .,NaN,NaN
4,333674203331051520,NaN,NaN,NaN,NaN,seroquel,"time for my daily afternoon relaxation ritual of smoking weed, taking 2 mgs of clonazepam, and 400 mg of seroquel xr.",NaN,NaN


In [ ]:
print(f"Shape of Training data: {training.shape}")
print(f"Shape of Validation data: {validation.shape}")

Shape of Training data: (2246, 9)
Shape of Validation data: (560, 9)


## 4. Prepare the data

In [ ]:
training.head()

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,331187619096588288,NaN,NaN,NaN,NaN,ofloxacin,@seefisch:oral drugs for pyelonephritis:ciprofloxacin levofloxacin tmp/smz do not use nitrofurantoin for pyelo(only cystitis)@david_medinaf,NaN,NaN
1,332227554956161024,NaN,NaN,NaN,NaN,trazodone,happy for wellbutrin; has similar effects as adderall.. trazodone is super promising for sleep.. but abilify can cause weight gain -_-,NaN,NaN
2,332448217490944000,NaN,NaN,NaN,NaN,lamotrigine,"@stilgarg i'm ok ty have an official diagnosis of bipolar now, feeling ok at the moment lamotrigine has been increased having monotherapy:/",NaN,NaN
3,332977955754110976,NaN,NaN,NaN,NaN,cymbalta,i'm soo depressed cymbalta couldn't help me .,NaN,NaN
4,333674203331051520,NaN,NaN,NaN,NaN,seroquel,"time for my daily afternoon relaxation ritual of smoking weed, taking 2 mgs of clonazepam, and 400 mg of seroquel xr.",NaN,NaN


In [ ]:
# Remove the rows which don't have ADR
# training_data = training[training.begin.notnull()]
# validation_data = validation[validation.begin.notnull()]

# Drop duplicate rows, only keep first
training.drop_duplicates(subset=["extraction", "tweet"], inplace=True, keep='first')
print(f"Shape after removing duplicates Training data: {training.shape}")

# Reset Index
training.reset_index(inplace=True, drop=True)

Shape after removing duplicates Training data: (2172, 9)


In [ ]:
# Pre-processing
# Referred from: https://github.com/cbaziotis/ekphrasis

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


In [ ]:
# Process the tweets
training['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in training.tweet]
validation['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in validation.tweet]

In [ ]:
# Create the label class
training['label'] = [1 if begin else 0 for begin in training.begin.notnull()]
validation['label'] = [1 if begin else 0 for begin in validation.begin.notnull()]

In [ ]:
# After pre-processing
training[['clean_tweets', 'label', 'extraction']].head()

,clean_tweets,label,extraction
0,<user> : oral drugs for pyelonephritis : ciprofloxacin levofloxacin tmp / smz do not use nitrofurantoin for pyelo ( only cystitis ) <user>,0,NaN
1,happy for wellbutrin ; has similar effects as adderall . <repeated> trazodone is super promising for sleep . <repeated> but abilify can cause weight gain -_-,0,NaN
2,"<user> i am ok ty have an official diagnosis of bipolar now , feeling ok at the moment lamotrigine has been increased having monotherapy <annoyed>",0,NaN
3,i am soo depressed cymbalta could not help me .,0,NaN
4,"time for my daily afternoon relaxation ritual of smoking weed , taking <number> mgs of clonazepam , and <number> mg of seroquel xr .",0,NaN


In [ ]:
validation[['clean_tweets', 'label', 'extraction']].head()

,clean_tweets,label,extraction
0,"do you have any medication allergies ? "" asthma ! <repeated> "" me : "" . <repeated> "" pt : "" no wait . avelox , that ' s it ! "" "" so no other allergies ? "" "" right ! "" * cont",1,allergies
1,"<user> if <hashtag> a velox </hashtag> has hurt your liver , avoid tylenol always , as it further damages liver , eat grapefruit unless taking cardiac drugs",1,HURT YOUR Liver
2,"apparently , baclofen greatly exacerbates the "" ad "" part of my adhd . average length of focus today : about <number> seconds .",1,AD
3,"apparently , baclofen greatly exacerbates the "" ad "" part of my adhd . average length of focus today : about <number> seconds .",1,focus
4,pt of mine died from cipro rt <user> : <user> if only more doctors thought like you ! i lost my entire life to <number> cipro pills,1,died


### Prepare tweets for RoBERTa

In [ ]:
BATCH_SIZE = 32
N_EPOCHS = 5
MAX_LENGTH = 128
LEARNING_RATE = 2e-5

In [ ]:
# Define tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# Tokenize train and validation data
training_enc = tokenizer.batch_encode_plus(training.clean_tweets.to_list(), padding="longest", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
validation_enc = tokenizer.batch_encode_plus(validation.clean_tweets.to_list(), padding="longest", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")

In [ ]:
training_enc.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
# Define dataloader
def get_dataloader(encoding, target, set='train'):
    if set == 'train':
        data = (TensorDataset(encoding.input_ids, encoding.attention_mask, target, torch.tensor(training.index.values.tolist())))
    else:
        data = (TensorDataset(encoding.input_ids, encoding.attention_mask, target, torch.tensor(validation.index.values.tolist())))
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader

In [ ]:
# Get train and validation dataloaders
training_dataloader = get_dataloader(training_enc, torch.tensor(training.label.to_list()), 'train')
validation_dataloader = get_dataloader(validation_enc, torch.tensor(validation.label.to_list()), 'valid')

In [ ]:
# Sanity check that the tensors returned by the dataloader are correct
for batch in training_dataloader:
    input_ids, attn_mask, target, index = batch
    print(input_ids.shape, attn_mask.shape, target.shape, index.shape)
    break

torch.Size([32, 124]) torch.Size([32, 124]) torch.Size([32]) torch.Size([32])


## 5. Model Building

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
class RobertaClassifier(nn.Module):
    def __init__(self, transformer):
        super(RobertaClassifier, self).__init__()
        self.transformer = transformer
        self.linear_layer = nn.Linear(768, 2)

    def forward(self, ip_ids, attn_mask):
        op = self.transformer(input_ids=ip_ids,
                              attention_mask=attn_mask)
        return self.linear_layer(op["pooler_output"])

In [ ]:
def count_parameter(model):
    return sum(para.numel() for para in model.parameters() if para.requires_grad)

In [ ]:
transformer = RobertaModel.from_pretrained('roberta-base')
model = RobertaClassifier(transformer).to(device)

In [ ]:
print(f"The model has {count_parameter(model):,} trainable parameters.")

The model has 124,647,170 trainable parameters.


In [ ]:
# Define optimizer 
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Define the train and validation functions

def train(model, dataloader, clip=1.0):
    model.train()

    epoch_loss = 0
    batch_num = 0
    y_pred, y_true = [], []

    for index, batch in tqdm(enumerate(dataloader)):
        batch = tuple(row.to(device) for row in batch)
        input_ids, attn_mask, target, indexes = batch

        optim.zero_grad()
        output = model(input_ids, attn_mask)
        loss = criterion(output, target)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optim.step()

        epoch_loss += loss.item()
        batch_num += 1
        y_pred.extend(torch.argmax(output, -1).tolist())
        y_true.extend(target.tolist())

    return epoch_loss/batch_num, f1_score(y_true, y_pred)

def evaluate(model, dataloader):
    model.eval()

    epoch_loss = 0
    batch_num = 0
    y_pred, y_true = [], []

    for index, batch in enumerate(dataloader):
        batch = tuple(row.to(device) for row in batch)
        input_ids, attn_mask, target, indexes = batch
        
        with torch.no_grad():
            output = model(input_ids, attn_mask)
            loss = criterion(output, target)
            
            epoch_loss += loss.item()
            batch_num += 1
            y_pred.extend(torch.argmax(output, -1).tolist())
            y_true.extend(target.tolist())
    
    return epoch_loss/batch_num, f1_score(y_true, y_pred), y_pred, y_true

In [ ]:
best_valid_loss = float('inf')
total_train_loss, total_valid_loss = list(), list()

In [ ]:
for epoch in tqdm(range(N_EPOCHS)):
    train_loss, train_f1_score = train(model, training_dataloader)
    total_train_loss.append(train_loss)

    valid_loss, valid_f1_score, pred, target = evaluate(model, validation_dataloader)
    total_valid_loss.append(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss_rob = valid_loss
        best_pred, best_target = pred, target
        torch.save(model.state_dict(), "model_least_loss_rob.pt")
        print("\nBest Model Saved!!\n")
    
    torch.save(model.state_dict(), "model_checkpoint_rob" + str(epoch+1) + ".pt")
    print("Checkpoint Model Saved!\n")

    print(f"Epoch: {epoch+1:02}")
    print(f"Train Total Loss: {train_loss:.3f} | Train F1 Score: {train_f1_score:.3f}")
    print(f"Valid Total Loss: {valid_loss:.3f} | Valid F1 Score: {valid_f1_score:.3f}")
    print("-"*20)

  0%|          | 0/5 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:01,  1.36s/it]
2it [00:02,  1.28s/it]
3it [00:03,  1.25s/it]
4it [00:05,  1.23s/it]
5it [00:06,  1.22s/it]
6it [00:07,  1.22s/it]
7it [00:08,  1.21s/it]
8it [00:09,  1.20s/it]
9it [00:11,  1.21s/it]
10it [00:12,  1.20s/it]
11it [00:13,  1.20s/it]
12it [00:14,  1.20s/it]
13it [00:15,  1.20s/it]
14it [00:17,  1.20s/it]
15it [00:18,  1.20s/it]
16it [00:19,  1.20s/it]
17it [00:20,  1.20s/it]
18it [00:21,  1.20s/it]
19it [00:23,  1.21s/it]
20it [00:24,  1.21s/it]
21it [00:25,  1.20s/it]
22it [00:26,  1.20s/it]
23it [00:27,  1.21s/it]
24it [00:29,  1.21s/it]
25it [00:30,  1.20s/it]
26it [00:31,  1.20s/it]
27it [00:32,  1.20s/it]
28it [00:33,  1.20s/it]
29it [00:35,  1.20s/it]
30it [00:36,  1.21s/it]
31it [00:37,  1.21s/it]
32it [00:38,  1.20s/it]
33it [00:39,  1.21s/it]
34it [00:41,  1.21s/it]
35it [00:42,  1.21s/it]
36it [00:43,  1.21s/it]
37it [00:44,  1.21s/it]
38it [00:45,  1.21s/it]
39it [00:47,  1.21s/it]
40it [00:48,  1.


Best Model Saved!!



 20%|██        | 1/5 [01:31<06:05, 91.26s/it]

Checkpoint Model Saved!

Epoch: 01
Train Total Loss: 0.106 | Train F1 Score: 0.975
Valid Total Loss: 0.767 | Valid F1 Score: 0.899
--------------------



0it [00:00, ?it/s]
1it [00:01,  1.34s/it]
2it [00:02,  1.29s/it]
3it [00:03,  1.25s/it]
4it [00:05,  1.24s/it]
5it [00:06,  1.26s/it]
6it [00:07,  1.25s/it]
7it [00:08,  1.24s/it]
8it [00:09,  1.23s/it]
9it [00:11,  1.22s/it]
10it [00:12,  1.22s/it]
11it [00:13,  1.22s/it]
12it [00:14,  1.22s/it]
13it [00:16,  1.21s/it]
14it [00:17,  1.21s/it]
15it [00:18,  1.22s/it]
16it [00:19,  1.22s/it]
17it [00:20,  1.22s/it]
18it [00:22,  1.22s/it]
19it [00:23,  1.21s/it]
20it [00:24,  1.21s/it]
21it [00:25,  1.21s/it]
22it [00:26,  1.22s/it]
23it [00:28,  1.21s/it]
24it [00:29,  1.21s/it]
25it [00:30,  1.21s/it]
26it [00:31,  1.21s/it]
27it [00:33,  1.22s/it]
28it [00:34,  1.21s/it]
29it [00:35,  1.21s/it]
30it [00:36,  1.21s/it]
31it [00:37,  1.21s/it]
32it [00:39,  1.21s/it]
33it [00:40,  1.22s/it]
34it [00:41,  1.22s/it]
35it [00:42,  1.22s/it]
36it [00:43,  1.22s/it]
37it [00:45,  1.22s/it]
38it [00:46,  1.22s/it]
39it [00:47,  1.21s/it]
40it [00:48,  1.21s/it]
41it [00:50,  1.21s/it]
42it 

...

Checkpoint Model Saved!

Epoch: 04
Train Total Loss: 0.026 | Train F1 Score: 0.995
Valid Total Loss: 0.862 | Valid F1 Score: 0.909
--------------------



0it [00:00, ?it/s]
1it [00:01,  1.33s/it]
2it [00:02,  1.28s/it]
3it [00:03,  1.25s/it]
4it [00:04,  1.23s/it]
5it [00:06,  1.22s/it]
6it [00:07,  1.22s/it]
7it [00:08,  1.22s/it]
8it [00:09,  1.22s/it]
9it [00:11,  1.22s/it]
10it [00:12,  1.22s/it]
11it [00:13,  1.21s/it]
12it [00:14,  1.21s/it]
13it [00:15,  1.21s/it]
14it [00:17,  1.21s/it]
15it [00:18,  1.22s/it]
16it [00:19,  1.21s/it]
17it [00:20,  1.21s/it]
18it [00:21,  1.21s/it]
19it [00:23,  1.21s/it]
20it [00:24,  1.22s/it]
21it [00:25,  1.21s/it]
22it [00:26,  1.22s/it]
23it [00:28,  1.22s/it]
24it [00:29,  1.22s/it]
25it [00:30,  1.23s/it]
26it [00:31,  1.23s/it]
27it [00:33,  1.23s/it]
28it [00:34,  1.23s/it]
29it [00:35,  1.22s/it]
30it [00:36,  1.22s/it]
31it [00:37,  1.22s/it]
32it [00:39,  1.22s/it]
33it [00:40,  1.22s/it]
34it [00:41,  1.22s/it]
35it [00:42,  1.22s/it]
36it [00:43,  1.21s/it]
37it [00:45,  1.21s/it]
38it [00:46,  1.21s/it]
39it [00:47,  1.21s/it]
40it [00:48,  1.21s/it]
41it [00:50,  1.21s/it]
42it 


Best Model Saved!!



100%|██████████| 5/5 [07:37<00:00, 91.53s/it]

Checkpoint Model Saved!

Epoch: 05
Train Total Loss: 0.028 | Train F1 Score: 0.996
Valid Total Loss: 0.918 | Valid F1 Score: 0.905
--------------------


In [ ]:
print(classification_report(best_target, best_pred))

              precision    recall  f1-score   support

           0       0.86      0.75      0.80       195
           1       0.87      0.94      0.90       365

    accuracy                           0.87       560
   macro avg       0.87      0.84      0.85       560
weighted avg       0.87      0.87      0.87       560



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Save the weights to drive
from glob import glob
import time
for filepath in glob("*.pt"):
    !cp -r $filepath /content/gdrive/My\ Drive/Colab\ Notebooks/NLP\ Final\ Project/
    time.sleep(10)

In [ ]:
# Loading the saved model
output_model = 'model_least_loss_rob.pt'

model_test = RobertaClassifier(transformer).to(device)
model_test.load_state_dict(torch.load(output_model, map_location=device))

<All keys matched successfully>

In [ ]:
# Training set

# Set model to evaluation
model_test.eval()

y_pred_train, y_true_train = [], []
train_indexes_list = []

for index, batch in enumerate(training_dataloader):
    batch = tuple(row.to(device) for row in batch)
    input_ids, attn_mask, target, indexes = batch
    
    with torch.no_grad():
        output = model_test(input_ids, attn_mask)
        
        y_pred_train.extend(torch.argmax(output, -1).tolist())
        y_true_train.extend(target.tolist())
        train_indexes_list.extend(indexes.tolist())

In [ ]:
print(f"F1-score: {f1_score(y_true_train, y_pred_train)}\n", f"Classification report: \n{classification_report(y_true_train, y_pred_train)}", sep='\n')

F1-score: 0.9996512033484479

Classification report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       739
           1       1.00      1.00      1.00      1433

    accuracy                           1.00      2172
   macro avg       1.00      1.00      1.00      2172
weighted avg       1.00      1.00      1.00      2172



In [ ]:
# Validation set
y_pred_valid, y_true_valid = [], []
valid_indexes_list = []

for index, batch in enumerate(validation_dataloader):
    batch = tuple(row.to(device) for row in batch)
    input_ids, attn_mask, target, indexes = batch
    
    with torch.no_grad():
        output = model_test(input_ids, attn_mask)
        
        y_pred_valid.extend(torch.argmax(output, -1).tolist())
        y_true_valid.extend(target.tolist())
        valid_indexes_list.extend(indexes.tolist())

In [ ]:
print(f"F1-score: {f1_score(y_true_valid, y_pred_valid)}\n", f"Classification report: \n{classification_report(y_true_valid, y_pred_valid)}", sep='\n')

F1-score: 0.9047619047619047

Classification report: 
              precision    recall  f1-score   support

           0       0.86      0.75      0.80       195
           1       0.87      0.94      0.90       365

    accuracy                           0.87       560
   macro avg       0.87      0.84      0.85       560
weighted avg       0.87      0.87      0.87       560



In [ ]:
training.loc[train_indexes_list, 'ADR'] = y_pred_train
validation.loc[valid_indexes_list, 'ADR'] = y_pred_valid

In [ ]:
training.to_csv("training_data_with_ADR.csv")
validation.to_csv("validation_data_with_ADR.csv")

In [ ]:
# Cross-check that it is correctly mapped
print(f"F1-score: {f1_score(training.label, training.ADR)}\n", f"Classification report: \n{classification_report(training.label, training.ADR)}", sep='\n')

F1-score: 0.9996512033484479

Classification report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       739
           1       1.00      1.00      1.00      1433

    accuracy                           1.00      2172
   macro avg       1.00      1.00      1.00      2172
weighted avg       1.00      1.00      1.00      2172

